<a href="https://www.kaggle.com/code/andrey36912/notebook8c9ac4360d?scriptVersionId=192531806" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch
from torchvision import datasets, transforms
from PIL import Image
from torch.utils.data import Dataset
device = "cuda" if torch.cuda.is_available() else "cpu"
data_dir = '/kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_0'

dataset = datasets.ImageFolder(root=data_dir)
!ls /kaggle/input/leukemia-classification/C-NMC_Leukemia/training_data/fold_0
all  hem
kaggle datasets download -d andrewmvd/leukemia-classification

transform = transforms.Compose([
    transforms.Resize((256, 256)), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.5], std=[0.5])
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)
train_ratio = 0.8
train_size = int(train_ratio * len(dataset))
val_size = len(dataset) - train_size

train_data, val_data = random_split(dataset, [train_size, val_size])
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Трансформації для тестових даних
test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])
class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_paths, self.labels = self.get_paths_and_labels()
        self.transform = transform

    def get_paths_and_labels(self):
        image_paths = []
        labels = []
        for label in os.listdir(self.root_dir):
            class_dir = os.path.join(self.root_dir, label)
            if os.path.isdir(class_dir):
                for img_name in os.listdir(class_dir):
                    img_path = os.path.join(class_dir, img_name)
                    image_paths.append(img_path)
                    labels.append(label)
        return image_paths, labels

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label_name = self.labels[idx]
        label = 0 if label_name == 'all' else 1  # Припускається бінарна класифікація
        if self.transform:
            image = self.transform(image)
        return image, label
custom_train_data = ImageDataset(data_dir, transform=train_transform)
custom_test_data = ImageDataset(data_dir, transform=test_transform)
print(f'Number of training samples: {len(custom_train_data)}')
print(f'Number of testing samples: {len(custom_test_data)}')
Number of training samples: 3527
Number of testing samples: 3527
import matplotlib.pyplot as plt

for i in range(3):  
    img, label = custom_train_data[i]


    img = img.numpy()

   
    img = img.transpose((1, 2, 0))

   
    label_name = 'all' if label == 0 else 'hem'

   
    plt.imshow(img)
    plt.title(f"Image: {i+1}, Label: {label_name}")
    plt.show()